## Assignment 1

In [39]:
import networkx as nx
import pandas as pd
import numpy as np
import random
import plotly.graph_objects as go


## Util Functions

In [14]:

def build_graph_from_df(df):
    # Get all videos with at least one valid pair
    valid_videos = df['videoID'].unique()

    # Get all users involved in those videos
    users_involved = pd.unique(df[['userID_1', 'userID_2']].values.ravel())

    G = nx.Graph()
    
    # Add user nodes
    for user in users_involved:
        G.add_node(user, type='user')

    # Add video nodes
    for video in valid_videos:
        G.add_node(video, type='video')


    # Getting user and videos dataframe
    user1_video_df = df[['videoID', 'userID_1']].drop_duplicates()
    user2_video_df = df[['videoID', 'userID_2']].drop_duplicates()
    user2_video_df.rename(columns={'userID_2': 'userID_1'}, inplace=True)
    user_video_df = pd.concat([user1_video_df, user2_video_df]).drop_duplicates()
    user_video_df.rename(columns={'userID_1': 'user', 'videoID': 'video'}, inplace=True)

    # Adding edges
    for _, row in user_video_df.iterrows():
        user = row['user']
        video = row['video']
        G.add_edge(user, video)

    # Getting user interaction
    user_interactions = df[['userID_1', 'userID_2']].drop_duplicates()

    # Adding edges
    for _, row in user_interactions.iterrows():
        user1 = row['userID_1']
        user2 = row['userID_2']
        G.add_edge(user1, user2)
    
    return G

In [23]:
def show_graph_metrics(G):
    
    # Number of nodes
    num_nodes = G.number_of_nodes()
    
    # Number of edges
    num_edges = G.number_of_edges()
    
    # Degree distribution
    degrees = [d for n, d in G.degree()]
    mean_degree = np.mean(degrees)
    var_degree = np.var(degrees)

    # Average clustering coefficient
    clustering = nx.average_clustering(G)

    # Components
    components = nx.connected_components(G)
    num_components = len(list(components))

    # Density
    density = nx.density(G)

    if nx.is_connected(G):
        # Average shortest path length
        avg_path_length = nx.average_shortest_path_length(G)
        # Diameter
        diameter = nx.diameter(G)

    else:
        avg_path_length = np.nan
        diameter = np.nan

    print(f"Number of nodes: {num_nodes}")    
    print(f"Number of edges: {num_edges}")
    print(f"Number of components: {num_components}")
    print(f"Mean degree: {mean_degree:.2f}")
    print(f"Degree variance: {var_degree:.2f}")
    print(f"Density: {density:.4f}")
    print(f"Diameter: {diameter}")
    print(f"Average clustering coefficient: {clustering:.4f}")
    print(f"Average path length: {avg_path_length:.4f}")

In [32]:
def transform_graph(G, connections, leave_larges_component = False):
    
    # Getting the list of video nodes (type="video")
    video_nodes = [n for n, d in G.nodes(data=True) if d["type"] == "video"]

    # Getting the list of user nodes (type="user")
    user_nodes = [n for n, d in G.nodes(data=True) if d["type"] == "user"]

    # Creating dataframe counting neighbors of each video node
    video_neighbors = pd.DataFrame(columns=["video", "user_count"])
    for video in video_nodes:
        video_neighbors.loc[video] = [video, len(list(G_transformed.neighbors(video)))]

    # Reseting index
    video_neighbors.reset_index(inplace=True)
    video_neighbors = video_neighbors.drop(columns=["index"])

    edges_added: int = 0
    # Adding connections
    for _ in range(connections):
        
        # Getting a video randomly, high changes with high number of user_count
        probabilities = video_neighbors['user_count'] / video_neighbors['user_count'].sum()
        random_index = np.random.choice(video_neighbors.index, p=probabilities)
        selected_video = video_neighbors.iloc[random_index]["video"]

        # Getting a user randomly from the video
        user = random.choice(user_nodes)

        # Validate that video is not connected to user
        if not G.has_edge(user, selected_video):
            # Getting user neighbors from user node that is type="user"
            user_neighbors = [n for n in G.neighbors(user) if G.nodes[n].get("type") == "user"]

            # Selecting one random user neighbor
            user_neighbor = random.choice(user_neighbors)

            # Validating if users are not connected
            if G.has_edge(user, video):
                continue

            if G.has_edge(user_neighbor, video):
                continue
            
            # Adding edge between user and video
            G.add_edge(user, video)
            G.add_edge(user_neighbor, video)
            edges_added += 1
    
    if leave_larges_component:
        # Get all connected components
        connected_components = list(nx.connected_components(G))

        # Find the largest component
        largest_component = max(connected_components, key=len)

        # Create a subgraph with only the largest component
        G = G.subgraph(largest_component).copy()

    print(f"Added {edges_added} edges")
    return G

In [37]:
def draw_degree_distribution(G):
    # Degree distribution
    degrees = [d for n, d in G.degree()]
    unique_degrees, counts = np.unique(degrees, return_counts=True)
    prob = counts / sum(counts)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=unique_degrees, 
        y=prob,
        mode='markers',
        marker=dict(size=6, color='royalblue'),
        name="Degree Distribution"
    ))

    fig.update_xaxes(type="log", title="Degree (k)")
    fig.update_yaxes(type="log", title="P(k)")
    fig.update_layout(
        title="Degree Distribution (Log-Log)",
        template="plotly_white"
    )

    fig.show()

## Analysis: Barabasi Albert Model
This is a network growth model that generates networks through a process called preferential attachment. It starts with a small number of nodes and adds new ones, with each new connection is based on the degree of the existing nodes. Nodes with higher degree are more likely to be connected to new nodes. The mechanism is commonly known to follow the principle "rich get richer".

## Load Real Data

In [1]:
# Loading the dataset of white helmets
import pandas as pd
df = pd.read_csv("pairwise_52seconds_share.csv")
df.head()

,Unnamed: 0,videoID,userID_1,userID_2,timestamp_1,timestamp_2,time_diff_seconds
0,0,-6bGXfM8-gs,19372991|840224732847833,19372991|840224732847833,2018-07-22 21:19:58,2018-07-22 21:19:58,0.0
1,10,-fJbMWhkTAw,Ej8Mm0YMadzmx4osDA_hgg,Ej8Mm0YMadzmx4osDA_hgg,2018-08-01 00:51:08,2018-08-01 00:51:08,0.0
2,11,-ilNuSh1Fgw,feNNP607aG1F64jR6bk8jw,CVEf5dB1MvNRTQFYivAIPQ,2018-04-27 22:28:49,2018-04-27 22:29:36,47.0
3,12,-ilNuSh1Fgw,5SDVRa-J-_cWYP6g0WNzLw,jz6hyweGgVHGTw-PbEMqKw,2018-05-14 16:52:08,2018-05-14 16:52:24,16.0
4,13,-ilNuSh1Fgw,42Egn_22OjOzg2XMqAa9_g,poH0yvIGbS5_7MdXM4EuRA,2018-05-14 16:55:04,2018-05-14 16:55:15,11.0


In [3]:
# Removing records where userID_1 is equal to userID_2
df = df[df['userID_1'] != df['userID_2']]
df.head()

,Unnamed: 0,videoID,userID_1,userID_2,timestamp_1,timestamp_2,time_diff_seconds
2,11,-ilNuSh1Fgw,feNNP607aG1F64jR6bk8jw,CVEf5dB1MvNRTQFYivAIPQ,2018-04-27 22:28:49,2018-04-27 22:29:36,47.0
3,12,-ilNuSh1Fgw,5SDVRa-J-_cWYP6g0WNzLw,jz6hyweGgVHGTw-PbEMqKw,2018-05-14 16:52:08,2018-05-14 16:52:24,16.0
4,13,-ilNuSh1Fgw,42Egn_22OjOzg2XMqAa9_g,poH0yvIGbS5_7MdXM4EuRA,2018-05-14 16:55:04,2018-05-14 16:55:15,11.0
5,14,-ilNuSh1Fgw,42Egn_22OjOzg2XMqAa9_g,MnOZgKSIq_7RKuR6XZqlxA,2018-05-14 16:55:04,2018-05-14 16:55:32,28.0
6,15,-ilNuSh1Fgw,poH0yvIGbS5_7MdXM4EuRA,MnOZgKSIq_7RKuR6XZqlxA,2018-05-14 16:55:15,2018-05-14 16:55:32,17.0


In [24]:
# Building the graph and showing some metrics
G = build_graph_from_df(df)

# Showing some metrics
show_graph_metrics(G)


Number of nodes: 4242
Number of edges: 8169
Number of components: 210
Mean degree: 3.85
Degree variance: 785.42
Density: 0.0009
Diameter: nan
Average clustering coefficient: 0.8822
Average path length: nan


## Transforming Graph
The graph generated with the real data is composed by many disconnected components. Let's tranform it into a connected graph taking the videos that are most connected to the rest of the graph.

In [33]:
G_transformed = G.copy()
G_transformed = transform_graph(G_transformed, connections = 20, leave_larges_component=True)
show_graph_metrics(G_transformed)


Added 17 edges
Number of nodes: 2245
Number of edges: 4849
Number of components: 1
Mean degree: 4.32
Degree variance: 1336.73
Density: 0.0019
Diameter: 6
Average clustering coefficient: 0.8839
Average path length: 3.3270


## Synthetic Barabasi Albert Model

In [34]:
# Creating a BA synthetic network
n = G_transformed.number_of_nodes()
avg_degree = G_transformed.number_of_edges() / n
m = int(round(avg_degree))
G_ba = nx.barabasi_albert_graph(n, m)
show_graph_metrics(G_ba)

Number of nodes: 2245
Number of edges: 4486
Number of components: 1
Mean degree: 4.00
Degree variance: 38.10
Density: 0.0018
Diameter: 8
Average clustering coefficient: 0.0182
Average path length: 4.3471


## Degree Distribution

In [40]:
draw_degree_distribution(G_transformed)

In [41]:
draw_degree_distribution(G_ba)